In [1]:
from datasets import load_from_disk

dataset = load_from_disk("/fs/nexus-projects/HuangWM/datasets/mscoco_5k")
images1 = [dataset[i]["image"] for i in range(0, 500)]
images2 = [dataset[i]["image"] for i in range(500, 1000)]

In [8]:
from metrics import *

print(
    compute_ssim_repeated(images1, images1, verbose=True),
    compute_ssim_repeated(images1, images2, verbose=True),
)

SSIM :   0%|          | 0/2500 [00:00<?, ?it/s]

KeyboardInterrupt: 